<a href="https://colab.research.google.com/github/Shirley-Dongxx/APS360_project/blob/main/APS360_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APS360 Project: Facial Age Progression and Regression


---
Contents to be added:


*   Image Preprocessing
*   CAAE Architecture
*   GAN Architecture
*   Baseline (optional)




## Data Preprocessing



*   Dataset used: The IMDB-WIKI dataset (only the WIKI set will be used)
*   Download the WIKI face only dataset at: https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/



### Import data from Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Load the mat file of the Wiki dataset


**Reference**

@article\
{Rothe-IJCV-2018,
  author = {Rasmus Rothe and Radu Timofte and Luc Van Gool},\
  title = {Deep expectation of real and apparent age from a single 
  image without facial landmarks},\
  journal = {International Journal of Computer Vision},\
  volume={126},\
  number={2-4},\
  pages={144--157},\
  year={2018},\
  publisher={Springer}
}

@InProceedings\
{Rothe-ICCVW-2015,\
  author = {Rasmus Rothe and Radu Timofte and Luc Van Gool},\
  title = {DEX: Deep EXpectation of apparent age from a single image},\
  booktitle = {IEEE International Conference on Computer Vision Workshops (ICCVW)},\
  year = {2015}, \
  month = {December},\
}



In [2]:
from datetime import datetime, timedelta
from scipy.io import loadmat
import numpy as np
import pandas as pd
import scipy

In [3]:
# convert the matlab styled date to number
def matlab_datenum_to_dt(matlab_datenum):
    return datetime.fromordinal(int(matlab_datenum) - 366) + timedelta(days=int(matlab_datenum % 1))

In [15]:
# load wiki metadata
# Modified from https://gist.github.com/messefor/e2ee5fe1c18a040c90bbf91f2ee279e3

def load_wiki_meta():

    # Please change to your own path when testing!
    file_path = "/content/gdrive/MyDrive/APS360_project/wiki.mat"
    save_path = "/content/gdrive/MyDrive/APS360_project/wiki.pkl"

    mat = loadmat(file_path)

    print("Data header:",mat['__header__'])
    print("Data Version:",mat['__version__'])

    # Extract values
    data = mat['wiki'][0, 0]
    print("Column names are: ", data.dtype.names)

    # Data loaded in simple form
    col_keys = ('dob', 'photo_taken', 'gender', 'face_location', 'face_score', 'second_face_score')
    col_values = {k: data[k].squeeze() for k in col_keys}

    # Data loaded into numpy arrays
    col_keys_nested = ('full_path', 'name')
    for key in col_keys_nested:
        col_values[key] = np.array([x if not x else x[0] for x in data[key][0]])
    #print(col_values['name'][5])

    # Convert face location to DataFrame
    # Inputs:
    #    img - image (i.e. load with imread)
    #    box - location of face (i.e. img(box(2):box(4),box(1):box(3),:))
    #    crop_margin - margin around face as a fraction of the width, height
    #    [left above right below], default is [0.4 0.4 0.4 0.4]
    col_values['face_location'] =[tuple(x[0].tolist()) for x in data['face_location'].squeeze()]

    # Check all values extracted have same length
    set_nrows = {len(v) for _, v in col_values.items()}
    assert len(set_nrows) == 1

    # convert to panda data frame
    df_values = pd.DataFrame(col_values)

    # Convert matlab datenum to datetime
    df_values['dob'] = df_values['dob'].apply(matlab_datenum_to_dt)

    # Calculate ages when photo was taken
    df_values['photo_taken_age'] = df_values.apply(lambda x: x['photo_taken'] - x['dob'].year, axis=1)

    # Concat all together and save
    # Do not use csv format to work around tuple to be string
    df_values.to_pickle(save_path)

load_wiki_meta()

Data header: b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Jan 16 16:25:20 2016'
Data Version: 1.0
Column names are:  ('dob', 'photo_taken', 'full_path', 'gender', 'name', 'face_location', 'face_score', 'second_face_score')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


### Load the image into different datasets with labels